* click.txt - все клики за ноябрь в приложении на событие. С переходом на внешнюю ссылку.
* region.txt - юзеры и их регионы (если известно, где живет пользователь)
* tickets.txt - купленные юзерами за ноябрь билеты + некоторые из них вернули.
* uniq.txt - уникальные даты запуска мобильного приложения пользователями за ноябрь
* users.txt - все пользователи, зарегистрированные до 1 декабря
* events/events.csv - все активные события Пушки
* events/events_pushka_accepted_30122021.csv - вообще все события Пушки, включая уже неактивные (завершенные или исключенные)
* events/organizations.csv - организации

In [16]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from sklearn.preprocessing import OrdinalEncoder
import re

In [17]:
USERS_FILE_PATH = './main_data/users.txt'
UNIQ_FILE_PATH = './main_data/uniq.txt'
REGION_FILE_PATH = './main_data/region.txt'

In [18]:
def users_preprocessing() -> pd.DataFrame:
    users_df = pd.read_csv(USERS_FILE_PATH, sep=';')
    uniq_df = pd.read_csv(UNIQ_FILE_PATH, sep=';')
    region_df = pd.read_csv(REGION_FILE_PATH, sep=';')
    ord_enc = OrdinalEncoder()
    
    users_df['create_date'] = pd.to_datetime([datetime.fromtimestamp(x).strftime('%Y-%m-%d') for x in users_df['create_date']])
    uniq_df['create_date'] = pd.to_datetime(uniq_df['create_date'])
    uniq_df['user_phone_details_id'] = uniq_df['user_phone_details'].apply(lambda x: 
                                                                          re.sub('(\..*)', '', x.replace('iOS ', '')) 
                                                                           if 'iOS' in x 
                                                                           else re.sub('\ \(.*', '', x))
    uniq_df['user_phone_details_id'] = uniq_df['user_phone_details_id'].apply(lambda x: 
                                                                          re.sub('(\..*)', '', x) 
                                                                           if 'Android' in x
                                                                          else x)
    uniq_df['type_phone'] = uniq_df['user_phone_details_id'].apply(lambda x: 
                                                                          x.replace('iOS,', '2').split()[0]
                                                                           if 'iOS' in x 
                                                                           else x.replace('Android,', '1').split()[0])
    users_df = users_df.merge(uniq_df, on = ['user_id'], how='left')
    users_df = users_df.merge(region_df, on = ['user_id'], how='left')
    
    users_df['user_birth'] = pd.to_datetime(users_df['user_birth'], yearfirst = True,
                                            infer_datetime_format = True)
    users_df['age'] = users_df['user_birth'].apply(lambda x: int(round((datetime.now() - x).days / 365.2425, 0)))
    users_df.loc[users_df.region.notna(), ['region_code']] = ord_enc.fit_transform(users_df['region'].dropna().values.reshape(-1, 1))
    
    users_df = users_df.rename(columns = {'create_date_x': 'account_create_date', 'create_date_y': 'app_open_date'})
    users_df['diff'] =  users_df['app_open_date'] - users_df['account_create_date']
    #users_df.drop(columns = ['user_birth', 'region'])
    
    
    return users_df

In [19]:
users = users_preprocessing()
users.head()

,account_create_date,user_id,user_birth,app_open_date,user_phone_details,buyer_mobile_phone,user_phone_details_id,type_phone,region,age,region_code,diff
0,2021-09-02,ee457fb8cd024e46452147d026e3ffc4,2003-08-14,2021-11-22,"iOS, iOS 14.5",6a4816fe848efbd8c8fb8e8760d7cbd1,"iOS, 14",2,Саратовская область,18,64.0,81 days
1,2021-10-02,bb491c13ad78710345565c953cd82b7b,2000-03-06,2021-11-02,"Android, 11 (SDK 30)",527d5ae346fa42566f25ec418e6ece9c,"Android, 11",1,Новосибирская область,22,35.0,31 days
2,2021-10-02,bb491c13ad78710345565c953cd82b7b,2000-03-06,2021-11-19,"Android, 11 (SDK 30)",527d5ae346fa42566f25ec418e6ece9c,"Android, 11",1,Новосибирская область,22,35.0,48 days
3,2021-10-02,bb491c13ad78710345565c953cd82b7b,2000-03-06,2021-11-25,"Android, 11 (SDK 30)",527d5ae346fa42566f25ec418e6ece9c,"Android, 11",1,Новосибирская область,22,35.0,54 days
4,2021-09-03,cb22ee56f531b5cebfa1455b2b425427,2000-12-14,NaT,NaN,NaN,NaN,NaN,Тюменская область,21,74.0,NaT
